In [ ]:
# Google Cloud Vision
# Initialize ImageAnnotatorClient
from PIL import Image, ImageDraw, ImageFont
import io, os, json
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "photo-manager-proto.json" 

with open("photo-manager-proto.json") as f:
    db = json.load(f)

from google.cloud import vision

client = vision.ImageAnnotatorClient()

# from ImageDraw.ImageColor.colormap, to draw with...
colors = ['red', 'green', 'blue', 'yellow', 'orange', 'peachpuff', 'purple']


In [ ]:
file_path = os.path.abspath('marstrand.jpg')
with open(file_path, "rb") as image_file:
    image_data = image_file.read()

In [ ]:
print("=== Cloud Vision...")
vision_image = vision.Image(content=image_data)

In [ ]:
print("=== Labels:")
response = client.label_detection(image=vision_image)
print(response)
#for label in label_results.label_annotations:
#    print(f'{label.description} - {label.score}')

In [ ]:
print('=== Safe search:')
response = client.safe_search_detection(image=vision_image)
print(response)

In [ ]:
print('=== Landmarks:')
response = client.landmark_detection(image=vision_image)
print(response)
landmarks = response.landmark_annotations

for landmark in landmarks:
    print(landmark.description)
    for location in landmark.locations:
        lat_lng = location.lat_lng
        print('Latitude {}'.format(lat_lng.latitude))
        print('Longitude {}'.format(lat_lng.longitude))

In [ ]:
print('=== Faces:')
response = client.face_detection(image=vision_image)

pillow_image = Image.open(io.BytesIO(image_data)).convert("RGBA")
image_draw = ImageDraw.Draw(pillow_image)
for i, face in enumerate(response.face_annotations):
    vertices = (['({},{})'.format(vertex.x, vertex.y) for vertex in face.bounding_poly.vertices]) 
    print(f'Face #{i}, bounds: {", ".join(vertices)}')
    print(face)
    xs = list(map(lambda v: v.x, face.bounding_poly.vertices))
    ys = list(map(lambda v: v.y, face.bounding_poly.vertices))
    image_draw.rectangle((min(xs), min(ys), max(xs), max(ys)), outline="blue", width=3)
        
from IPython import display
# pillow_image.thumbnail((512,512))
# display.display(pillow_image)

# for mark in face.landmarks:
#     print(f'{mark.type_}({mark.position.x},{mark.position.y},{mark.position.z})')

In [ ]:
def display_image():
    for i, face in enumerate(response.face_annotations):
        vertices = (['({},{})'.format(vertex.x, vertex.y) for vertex in face.bounding_poly.vertices]) 
        print(f'Face #{i}, bounds: {", ".join(vertices)}')
        print(face)
        xs = list(map(lambda v: v.x, face.bounding_poly.vertices))
        ys = list(map(lambda v: v.y, face.bounding_poly.vertices))
        image_draw.rectangle((min(xs), min(ys), max(xs), max(ys)), outline="blue", width=3)
            

In [ ]:

def face_detect():
    print("=== Face API: Detect")

    import http.client, urllib.request, urllib.parse, urllib.error, json 

    headers = {
        # Request headers
        # 'Content-Type': 'application/json',
        'Content-Type': 'application/octet-stream',
        "Ocp-Apim-Subscription-Key": db["key1"],
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'returnFaceId': 'true',
        'returnFaceLandmarks': 'true',
        'returnFaceAttributes': 'age,smile,facialHair,glasses,emotion,hair,makeup,accessories',
        'recognitionModel': 'recognition_04',
        'returnRecognitionModel': 'true',
        'detectionModel': 'detection_01',
        'faceIdTimeToLive': '86400',
    })

    endpoint = db['uri']
    conn = http.client.HTTPSConnection(endpoint)
    conn.request("POST", "/face/v1.0/detect?%s" % params, body=image_data, headers=headers)
    response = conn.getresponse()
    data = response.read()
    conn.close()

    pillow_image = Image.open(io.BytesIO(image_data)).convert("RGBA")
    image_draw = ImageDraw.Draw(pillow_image)

    for i, face in enumerate(json.loads(data)):
        face_rectangle = face["faceRectangle"]
        text = f'\nFace {i}\n\tLocation: {face_rectangle}\n\tAge: {face["faceAttributes"]["age"]}\n\tEmotions: {face["faceAttributes"]["emotion"]}'
        print(text)
        image_draw.rectangle((face_rectangle['left'], face_rectangle['top'], 
                face_rectangle['left']+face_rectangle['width'], face_rectangle['top']+face_rectangle['height']), outline=colors[i%len(colors)], width=3)
        # image_draw.multiline_text((face_rectangle['left'], face_rectangle['top']), text, font=ImageFont.truetype("tahoma.ttf", size=48))

    from IPython import display
    # pillow_image.thumbnail((1024, 1024))
    display.display(pillow_image)

face_detect()

In [ ]:
def vision_detect():
    print("=== Vision API: Describe")
    import http.client, urllib.request, urllib.parse, urllib.error, pprint

    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        "Ocp-Apim-Subscription-Key": db["key1"],
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'maxCandidates': '3',
        'language': 'en',
        'model-version': 'latest',
    })

    endpoint = db['uri']
    conn = http.client.HTTPSConnection(endpoint)
    conn.request("POST", "/vision/v3.2/describe?%s" % params, body=image_data, headers=headers)
    response = conn.getresponse()
    data = response.read()
    pprint.pprint(json.loads(data))
    conn.close()

vision_detect()

In [ ]:
def vision_analyze():
    print("=== Vision API: Analyze")
    import http.client, urllib.request, urllib.parse, urllib.error, base64, pprint

    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        "Ocp-Apim-Subscription-Key": db["key1"],
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'visualFeatures': 'Categories,Adult,Tags,Brands,Color,Description,Faces,ImageType,Objects',
        'details': 'Celebrities,Landmarks',
        'language': 'en',
        'model-version': 'latest',
    })
    
    endpoint = db['uri']
    conn = http.client.HTTPSConnection(endpoint)
    conn.request("POST", "/vision/v3.2/analyze?%s" % params, body=image_data, headers=headers)
    response = conn.getresponse()
    data = response.read()   
    pprint.pprint(json.loads(data))
    conn.close()

vision_analyze()